In [ ]:
import numpy as np

import pandas as pd
from util import get_data, plot_data


def compute_portvals(
    orders_file="./orders/orders.csv",
    start_val=1000000,
    commission=9.95,
    impact=0.005,
):
    data = {"Date": [], "Cash": [], "Equity": [], "Position": []}

    orders = pd.read_csv(orders_file)
    start, end, period = get_dates(orders)

    symbols = orders.Symbol.unique()
    transaction_dates = orders.Date.unique()
    prices = get_prices(symbols, period)

    curr_cash = start_val
    shares_held = {}  # {key: symbol, val: shares}

    for date in prices.index:
        date = str(date)[:10]

        data["Date"].extend([date])
        data["Cash"].extend([curr_cash])
        data["Position"].extend([shares_held])
        curr_equity = get_equity_value(date, shares_held, prices)
        data["Equity"].extend([curr_equity])

        if date in transaction_dates:
            transactions = orders[orders.Date == date]
            for row in transactions.iterrows():
                date = row[1]["Date"]
                symbol = row[1]["Symbol"]
                action = row[1]["Order"]
                shares = row[1]["Shares"]
                price = prices.loc[date, symbol]
                curr_cash, shares_held = transact(
                    symbol,
                    action,
                    shares,
                    curr_cash,
                    commission,
                    impact,
                    price,
                    shares_held,
                )

    port_df = pd.DataFrame.from_dict(data)
    port_df["port"] = port_df["Cash"] + port_df["Equity"]

    return pd.DataFrame(port_df["port"])


def get_dates(orders):
    start = orders.Date.iloc[0]
    end = orders.Date.iloc[-1]
    period = pd.date_range(start, end)
    return start, end, period


def get_prices(symbols, period):
    return get_data(symbols, period)


def transact(symbol, action, shares, curr_cash, commission, impact, price, shares_held):
    if action == "BUY":
        shares_held[symbol] = shares_held.get(symbol, 0) + shares
        curr_cash = curr_cash - (price * (1 - impact) * shares - commission)
    elif action == "SELL":
        shares_held[symbol] = shares_held.get(symbol, 0) - shares
        curr_cash = curr_cash + (price * (1 - impact) * shares - commission)
    return curr_cash, shares_held


def get_equity_value(date, shares_held, prices):
    equity = 0
    for symbol in shares_held:
        equity += prices.loc[date, symbol] * shares_held[symbol]
    return equity


def test_code():
    of = "./orders/orders-09.csv"
    sv = 1000000

    portvals = compute_portvals(orders_file=of, start_val=sv)
    if isinstance(portvals, pd.DataFrame):
        portvals = portvals[portvals.columns[0]]  # just get the first column
    else:
        "warning, code did not return a DataFrame"

    print(portvals)


if __name__ == "__main__":
    test_code()


def author():
    return "yzhang3946"

In [46]:
orders = pd.read_csv("./orders/orders-09.csv")

In [47]:
orders

,Date,Symbol,Order,Shares
0,2011-01-10,AAPL,BUY,2500
1,2011-01-10,AAPL,BUY,2500
2,2011-01-13,AAPL,SELL,5000
3,2011-01-13,IBM,BUY,4000
4,2011-01-26,GOOG,BUY,1000
5,2011-02-02,XOM,SELL,4000
6,2011-02-10,XOM,BUY,4000
7,2011-03-03,GOOG,SELL,1000
8,2011-03-03,IBM,SELL,4000


In [48]:
start, end, period = get_dates(orders)

In [49]:
period

DatetimeIndex(['2011-01-10', '2011-01-11', '2011-01-12', '2011-01-13',
               '2011-01-14', '2011-01-15', '2011-01-16', '2011-01-17',
               '2011-01-18', '2011-01-19', '2011-01-20', '2011-01-21',
               '2011-01-22', '2011-01-23', '2011-01-24', '2011-01-25',
               '2011-01-26', '2011-01-27', '2011-01-28', '2011-01-29',
               '2011-01-30', '2011-01-31', '2011-02-01', '2011-02-02',
               '2011-02-03', '2011-02-04', '2011-02-05', '2011-02-06',
               '2011-02-07', '2011-02-08', '2011-02-09', '2011-02-10',
               '2011-02-11', '2011-02-12', '2011-02-13', '2011-02-14',
               '2011-02-15', '2011-02-16', '2011-02-17', '2011-02-18',
               '2011-02-19', '2011-02-20', '2011-02-21', '2011-02-22',
               '2011-02-23', '2011-02-24', '2011-02-25', '2011-02-26',
               '2011-02-27', '2011-02-28', '2011-03-01', '2011-03-02',
               '2011-03-03'],
              dtype='datetime64[ns]', freq='D')

In [25]:
orders

,Date,Symbol,Order,Shares
0,2011-01-10,AAPL,BUY,2500
1,2011-01-10,AAPL,BUY,2500
2,2011-01-13,AAPL,SELL,5000
3,2011-01-13,IBM,BUY,4000
4,2011-01-26,GOOG,BUY,1000
5,2011-02-02,XOM,SELL,4000
6,2011-02-10,XOM,BUY,4000
7,2011-03-03,GOOG,SELL,1000
8,2011-03-03,IBM,SELL,4000


In [124]:
for i in orders.index: 
    print(i)

0
1
2
3
4
5
6
7
8


In [126]:
orders.iloc[0]

Date      2011-01-10
Symbol          AAPL
Order            BUY
Shares          2500
Name: 0, dtype: object

In [85]:
symbols = orders.Symbol.unique()

In [36]:
symbols = list(symbols) + ['Cash']

In [50]:
df_price= pd.DataFrame(0, index=period, columns=symbols)

In [65]:
df_trading = get_prices(["$SPX"], period)

In [67]:
df_trading.index

DatetimeIndex(['2011-01-10', '2011-01-11', '2011-01-12', '2011-01-13',
               '2011-01-14', '2011-01-18', '2011-01-19', '2011-01-20',
               '2011-01-21', '2011-01-24', '2011-01-25', '2011-01-26',
               '2011-01-27', '2011-01-28', '2011-01-31', '2011-02-01',
               '2011-02-02', '2011-02-03', '2011-02-04', '2011-02-07',
               '2011-02-08', '2011-02-09', '2011-02-10', '2011-02-11',
               '2011-02-14', '2011-02-15', '2011-02-16', '2011-02-17',
               '2011-02-18', '2011-02-22', '2011-02-23', '2011-02-24',
               '2011-02-25', '2011-02-28', '2011-03-01', '2011-03-02',
               '2011-03-03'],
              dtype='datetime64[ns]', freq=None)

In [88]:
df_price = get_prices(symbols, period)

In [90]:
df_price['Cash'] = np.ones(df_price.shape[0])

In [92]:
df_price.head()

,SPY,AAPL,IBM,GOOG,XOM,Cash
2011-01-10,123.19,340.99,143.41,614.21,72.02,1.0
2011-01-11,123.63,340.18,143.06,616.01,72.56,1.0
2011-01-12,124.74,342.95,144.82,616.87,73.41,1.0
2011-01-13,124.54,344.20,144.55,616.69,73.54,1.0
2011-01-14,125.44,346.99,145.70,624.18,74.62,1.0


In [94]:
df_trades = df_price.copy()

In [117]:
df_trade= pd.DataFrame().reindex_like(df_price).fillna(0)

In [118]:
df_trade.head()

,SPY,AAPL,IBM,GOOG,XOM,Cash
2011-01-10,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-11,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-12,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-13,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-14,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
df_price = df_price.join(df_trading, how='right').iloc[:,:-2]

In [100]:
orders.head()

,Date,Symbol,Order,Shares
0,2011-01-10,AAPL,BUY,2500
1,2011-01-10,AAPL,BUY,2500
2,2011-01-13,AAPL,SELL,5000
3,2011-01-13,IBM,BUY,4000
4,2011-01-26,GOOG,BUY,1000


In [108]:
date = orders.iloc[0]['Date']

In [109]:
symbol = orders.iloc[0]['Symbol']

In [114]:
order = orders.iloc[0]['Order']

In [120]:
shares = orders.iloc[0]['Shares']

In [ ]:
shares = 
    (if order =="BUY": 
        orders.iloc[0]['Shares'],
    else: - orders.iloc[0]['Shares'])

In [121]:
df_trade.loc[date, symbol] = df_trade.loc[date, symbol] + shares 

In [ ]:
#calculate cash impact 
price = df_price.loc[date, symbol]
if order == "BUY": 
    cash_trade = - (price * (1- impact) * shares ) - commission 
else: 
    cash_trade =  (price * (1- impact) * shares ) - commission 

In [122]:
df_trade.head()

,SPY,AAPL,IBM,GOOG,XOM,Cash
2011-01-10,0.0,2500.0,0.0,0.0,0.0,0.0
2011-01-11,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-12,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-13,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-14,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
start = orders.Date.iloc[0]

In [22]:
import pandas as pd
import numpy as np
import datetime as dt
import os
from util import get_data, plot_data


def compute_portvals(
    orders_file="./orders/orders.csv", start_val=1000000, commission=9.95, impact=0.005
):
    # this is the function the autograder will call to test your code
    # NOTE: orders_file may be a string, or it may be a file object. Your
    # code should work correctly with either input
    # TODO: Your code here

    ##### setting up
    orders_df = pd.read_csv(
        orders_file, index_col="Date", parse_dates=True, na_values=["nan"]
    )
    start_date, end_date, orders_dates = get_dates(orders_df)

    # wrong
    portvals = get_data(
        ["SPY"], pd.date_range(start_date, end_date), addSPY=True, colname="Adj Close"
    )
    portvals = portvals.rename(columns={"SPY": "value"})
    dates = portvals.index

    ##### my account
    current_cash = start_val
    shares_owned = {}  # symbol (str) -> number (int)
    symbol_table = {}  # symbol (str) -> prices (pd.df)

    ##### going through dates
    for date in dates:
        if date in orders_dates:
            # date is <class 'pandas._libs.tslibs.timestamps.Timestamp'>
            # details is <class 'pandas.core.series.Series'>
            details = orders_df.loc[date]

            # if there are multiple orders on the same day
            if isinstance(details, pd.DataFrame):
                for _, each in details.iterrows():
                    symbol = each.loc["Symbol"]
                    order = each.loc["Order"]
                    shares = each.loc["Shares"]
                    current_cash, shares_owned, symbol_table = update_share_cash(
                        symbol,
                        order,
                        shares,
                        current_cash,
                        shares_owned,
                        symbol_table,
                        date,
                        end_date,
                        commission,
                        impact,
                    )
            # if there is only one order on that day
            else:
                symbol = details.loc["Symbol"]
                order = details.loc["Order"]
                shares = details.loc["Shares"]

                current_cash, shares_owned, symbol_table = update_share_cash(
                    symbol,
                    order,
                    shares,
                    current_cash,
                    shares_owned,
                    symbol_table,
                    date,
                    end_date,
                    commission,
                    impact,
                )

        ### calculating current protfolio value
        portvals.loc[date].loc["value"] = compute_portval(
            date, current_cash, shares_owned, symbol_table
        )

    return portvals


"""########
Helper functions for compute_portvals
"""  ########


# returns the start_date, end_date and orders_dates
def get_dates(orders_df):
    orders_dates = orders_df.index
    start_date = orders_df.index[0]
    end_date = orders_df.index[-1]
    return start_date, end_date, orders_dates


# update current_cash and shares_owned from an order
def update_share_cash(
    symbol,
    order,
    shares,
    current_cash,
    shares_owned,
    symbol_table,
    curr_date,
    end_date,
    commission,
    impact,
):
    # if we have not loaded the symbol information yet
    if symbol not in symbol_table:
        # get the df for the symbol
        symbol_df = get_data(
            [symbol],
            pd.date_range(curr_date, end_date),
            addSPY=True,
            colname="Adj Close",
        )
        # back fill and forward fill missing informations on market opend dates
        symbol_df = symbol_df.ffill().bfill()
        # add the symbol df to symbol_table
        symbol_table[symbol] = symbol_df
        # print(type(symbol_table[symbol]))

    # update the share and cash information
    if order == "BUY":
        share_change = shares
        cash_change = (
            -symbol_table[symbol].loc[curr_date].loc[symbol] * (1 + impact) * shares
        )
    elif order == "SELL":
        share_change = -shares
        cash_change = (
            symbol_table[symbol].loc[curr_date].loc[symbol] * (1 - impact) * shares
        )
    else:
        print("ERROR: unknow order type")

    shares_owned[symbol] = shares_owned.get(symbol, 0) + share_change
    current_cash += cash_change - commission

    return current_cash, shares_owned, symbol_table


# compute the portfolio value for a day
def compute_portval(curr_date, current_cash, shares_owned, symbol_table):
    shares_worth = 0
    for symbol in shares_owned:
        shares_worth += (
            symbol_table[symbol].loc[curr_date].loc[symbol] * shares_owned[symbol]
        )
    return current_cash + shares_worth



def test_code():

    of = "./orders/orders-09.csv"
    sv = 1000000

    # Process orders
    portvals = compute_portvals(orders_file=of, start_val=sv)
    if isinstance(portvals, pd.DataFrame):
        portvals = portvals[portvals.columns[0]]  # just get the first column
    else:
        "warning, code did not return a DataFrame"

    print(portvals)


def author():
    return "jlyu31"


if __name__ == "__main__":
    test_code()

2011-01-10     991455.35
2011-01-11     987405.35
2011-01-12    1001255.35
2011-01-13     995989.45
2011-01-14    1000589.45
2011-01-18    1003109.45
2011-01-19    1022669.45
2011-01-20    1023109.45
2011-01-21    1021949.45
2011-01-24    1037989.45
2011-01-25    1045029.45
2011-01-26    1040377.00
2011-01-27    1040787.00
2011-01-28    1017747.00
2011-01-31    1027957.00
2011-02-01    1044717.00
2011-02-02    1043067.85
2011-02-03    1041977.85
2011-02-04    1045247.85
2011-02-07    1049237.85
2011-02-08    1062647.85
2011-02-09    1056967.85
2011-02-10    1050614.30
2011-02-11    1057714.30
2011-02-14    1058924.30
2011-02-15    1053444.30
2011-02-16    1055674.30
2011-02-17    1059994.30
2011-02-18    1067174.30
2011-02-22    1036024.30
2011-02-23    1030214.30
2011-02-24    1030034.30
2011-02-25    1037134.30
2011-02-28    1038934.30
2011-03-01    1018854.30
2011-03-02    1019604.30
2011-03-03    1035078.20
Name: value, dtype: float64


In [13]:
start

'2011-01-10'

In [11]:
end = orders.Date.iloc[-1]

In [15]:
import datetime
a = datetime(start)
b = datetime.strptime(end)

delta = b - a

print(delta.days) 


TypeError: 'module' object is not callable

In [140]:
'2011-01-10' in orders.Date.unique()

True

In [153]:
prices.index[0]

Timestamp('2011-01-10 00:00:00')

In [123]:
start = orders.Date.iloc[0]
end = orders.Date.iloc[-1]
period = pd.date_range(start, end)

In [158]:
import numpy as np

import pandas as pd
from util import get_data, plot_data


def compute_portvals(
    orders_file="./orders/orders.csv",
    start_val=1000000,
    commission=9.95,
    impact=0.005,
):
    data = {"Date": [], "Cash": [], "Equity": [], "Position": []}

    orders = pd.read_csv(orders_file)
    start, end, period = get_dates(orders)

    symbols = orders.Symbol.unique()
    transaction_dates = orders.Date.unique()
    prices = get_prices(symbols, period)

    curr_cash = start_val
    shares_held = {}  # {key: symbol, val: shares}

    for date in prices.index:
        date = str(date)[:10]
        
        data["Date"].extend([date])
        data["Cash"].extend([curr_cash])
        data["Position"].extend([shares_held])
        curr_equity = get_equity_value(date, shares_held, prices)
        data["Equity"].extend([curr_equity])

        
        if date in transaction_dates: 
            transactions = orders[orders.Date == date]
            for row in transactions.iterrows():
                date = row[1]["Date"]
                symbol = row[1]["Symbol"]
                action = row[1]["Order"]
                shares = row[1]["Shares"]
                price = prices.loc[date, symbol]
                curr_cash, shares_held = transact(
                    symbol,
                    action,
                    shares,
                    curr_cash,
                    commission,
                    impact,
                    price,
                    shares_held,
                )

    port_df = pd.DataFrame.from_dict(data)
    port_df["port"] = port_df["Cash"] + port_df["Equity"]

    return pd.DataFrame(port_df["port"])


def get_dates(orders):
    start = orders.Date.iloc[0]
    end = orders.Date.iloc[-1]
    period = pd.date_range(start, end)
    return start, end, period


def get_prices(symbols, period):
    return get_data(symbols, period)


def transact(
    symbol, action, shares, curr_cash, commission, impact, price, shares_held
):
    if action == "BUY":
        shares_held[symbol] = shares_held.get(symbol, 0) + shares
        curr_cash = curr_cash - (price * (1 - impact) * shares - commission)
    elif action == "SELL":
        shares_held[symbol] = shares_held.get(symbol, 0) - shares
        curr_cash = curr_cash + (price * (1 - impact) * shares - commission)
    return curr_cash, shares_held


def get_equity_value(date, shares_held, prices):
    equity = 0
    for symbol in shares_held:
        equity += prices.loc[date, symbol] * shares_held[symbol]
    return equity


In [2]:
import pandas as pd
import numpy as np 
import datetime as dt  		   	  			  	 		  		  		    	 		 		   		 		  
import os  		   	  			  	 		  		  		    	 		 		   		 		  
from util import get_data, plot_data  	

In [84]:
orders_df = pd.read_csv(order)

In [85]:
orders_df

,Date,Symbol,Order,Shares
0,2011-01-10,AAPL,BUY,1500
1,2011-01-13,AAPL,SELL,1500
2,2011-01-13,IBM,BUY,4000
3,2011-01-26,GOOG,BUY,1000
4,2011-02-02,XOM,SELL,4000
5,2011-02-10,XOM,BUY,4000
6,2011-03-03,GOOG,SELL,1000
7,2011-03-03,GOOG,SELL,2200
8,2011-06-03,IBM,SELL,3300
9,2011-05-03,IBM,BUY,1500


In [91]:
for i in orders_df.iterrows(): 
    print(i[1]['Date'])

2011-01-10
2011-01-13
2011-01-13
2011-01-26
2011-02-02
2011-02-10
2011-03-03
2011-03-03
2011-06-03
2011-05-03
2011-06-10
2011-08-01
2011-08-01
2011-12-20
2011-12-21
2011-12-27
2011-12-28


In [88]:
orders_df[orders_df.Date == '2011-02-10']

,Date,Symbol,Order,Shares
5,2011-02-10,XOM,BUY,4000


In [81]:
orders_df.loc['2011-02-10']

Symbol     XOM
Order      BUY
Shares    4000
Name: 2011-02-10, dtype: object

In [61]:
symbols = orders_df['Symbol'].unique()

In [70]:
symbols[1]

'IBM'

In [45]:
for order in orders_df[:1].iterrows(): 
    a = order

In [67]:
a[0]

'2011-01-10'

In [71]:
shared_held = {}
shared_held.get(symbols[1],0)

0

In [77]:
end

Timestamp('2011-12-28 00:00:00')

In [75]:
for date in pd.date_range(start, end): 
    print(str(date)[:10])

2011-01-10
2011-01-11
2011-01-12
2011-01-13
2011-01-14
2011-01-15
2011-01-16
2011-01-17
2011-01-18
2011-01-19
2011-01-20
2011-01-21
2011-01-22
2011-01-23
2011-01-24
2011-01-25
2011-01-26
2011-01-27
2011-01-28
2011-01-29
2011-01-30
2011-01-31
2011-02-01
2011-02-02
2011-02-03
2011-02-04
2011-02-05
2011-02-06
2011-02-07
2011-02-08
2011-02-09
2011-02-10
2011-02-11
2011-02-12
2011-02-13
2011-02-14
2011-02-15
2011-02-16
2011-02-17
2011-02-18
2011-02-19
2011-02-20
2011-02-21
2011-02-22
2011-02-23
2011-02-24
2011-02-25
2011-02-26
2011-02-27
2011-02-28
2011-03-01
2011-03-02
2011-03-03
2011-03-04
2011-03-05
2011-03-06
2011-03-07
2011-03-08
2011-03-09
2011-03-10
2011-03-11
2011-03-12
2011-03-13
2011-03-14
2011-03-15
2011-03-16
2011-03-17
2011-03-18
2011-03-19
2011-03-20
2011-03-21
2011-03-22
2011-03-23
2011-03-24
2011-03-25
2011-03-26
2011-03-27
2011-03-28
2011-03-29
2011-03-30
2011-03-31
2011-04-01
2011-04-02
2011-04-03
2011-04-04
2011-04-05
2011-04-06
2011-04-07
2011-04-08
2011-04-09
2011-04-10

In [63]:
start_date = dt.datetime(2008, 1, 1)  		  	   		  	  			  		 			 	 	 		 		 	 		 		 	 		  	 	 			  	 
end_date = dt.datetime(2008, 6, 1) 
prices = get_data(symbols, pd.date_range(start_date, end_date))

In [68]:
port_val_df = pd.DataFrame(pd.date_range(start_date, end_date), columns=['date'])

In [69]:
port_val_df

,date
0,2008-01-01
1,2008-01-02
2,2008-01-03
3,2008-01-04
4,2008-01-05
5,2008-01-06
6,2008-01-07
7,2008-01-08
8,2008-01-09
9,2008-01-10


In [66]:
prices.loc['2008-01-10', 'AAPL']


177.26

In [7]:
orders_df.index

DatetimeIndex(['2011-01-10', '2011-01-13', '2011-01-13', '2011-01-26',
               '2011-02-02', '2011-02-10', '2011-03-03', '2011-03-03',
               '2011-06-03', '2011-05-03', '2011-06-10', '2011-08-01',
               '2011-08-01', '2011-12-20', '2011-12-21', '2011-12-27',
               '2011-12-28'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [8]:
def get_period(orders_df): 
    orders_dates = orders_df.index
    start = orders_dates[0]
    end = orders_dates[-1]
    return start, end, orders_dates

In [12]:
start, end, order_dates = get_period(orders_df)

In [16]:
start


Timestamp('2011-01-10 00:00:00')

In [9]:
current_cash = start_val = 1000000, 

In [ ]:
shares = {}
symbol = {}

In [28]:
details = orders_df.loc['2011-01-10']

In [29]:
details

,Symbol,Order,Shares
Date,,,
2011-01-10,AAPL,BUY,1500


In [20]:
[i for i in details.iterrows()]

[(Timestamp('2011-01-10 00:00:00'),
  Symbol    AAPL
  Order      BUY
  Shares    1500
  Name: 2011-01-10 00:00:00, dtype: object)]

In [34]:
date = '2011-01-10'

In [35]:
shares = details.loc[date, 'Shares']
symbol = details.loc[date, 'Symbol']
order = details.loc[date, 'Order']

In [ ]:
if order == 'BUY': 
    share_change = shares 
    cash_change = - price[symbol].loc[date] * (1 + impact ) * shares 

elif order == "SELL": 
    share_change = - shares 
    cash_change = price[symbol].loc[date] * (1 + impact ) * shares 

shares[symbol] = shares.get(symbol, 0) + share_change
current_cash += cash_change - commission

return current_cash, shares, price 

In [ ]:
def port_val(curr_date, current_cash, shares, symbol): 
    port = 0 
    for symbol in shares_owned: 
        port += price[symbol].loc[curr_date, symbol] * shares[symbol]
    return current_cash + port 

In [161]:
def compute_portvals(
    orders_file="./orders/orders-09.csv",
    start_val=1000000,
    commission=0,
    impact=0,
): 

    orders = pd.read_csv(orders_file)
    start, end, period = get_dates(orders)

    symbols = orders.Symbol.unique()
    df_price = get_prices(symbols, period)
    df_price['Cash'] = np.ones(df_price.shape[0])

    df_trade = pd.DataFrame().reindex_like(df_price).fillna(0)

    for i in orders.index: 

        date = orders.iloc[i]['Date']
        symbol = orders.iloc[i]['Symbol']
        order = orders.iloc[i]['Order']
        shares = orders.iloc[i]['Shares']
        price = df_price.loc[date, symbol]

        if order == "SELL": 
            shares = - shares 
            cash_trade = - (price * (1 - impact) * shares ) - commission 
        else: 
            cash_trade = - (price * (1 + impact) * shares ) - commission 

        df_trade.loc[date, symbol] += shares
        df_trade.loc[date, 'Cash'] += cash_trade

    df_holdings = pd.DataFrame().reindex_like(df_price).fillna(0)

    for i in range(df_holdings.shape[0]): 

        if i == 0: 
            df_holdings.iloc[i] = df_trade.iloc[i]
            df_holdings.iloc[i]['Cash'] += start_val

        else: 
            df_holdings.iloc[i] = df_holdings.iloc[i - 1] + df_trade.iloc[i]

    df_value = df_price * df_holdings
    portvals = df_value.sum(axis=1)
    
    return portvals 

In [136]:
df_price, df_trade = compute_portvals()

In [154]:
df_price.head()

,SPY,AAPL,IBM,GOOG,XOM,Cash
2011-01-10,123.19,340.99,143.41,614.21,72.02,1.0
2011-01-11,123.63,340.18,143.06,616.01,72.56,1.0
2011-01-12,124.74,342.95,144.82,616.87,73.41,1.0
2011-01-13,124.54,344.20,144.55,616.69,73.54,1.0
2011-01-14,125.44,346.99,145.70,624.18,74.62,1.0


In [155]:
340.99 * 5000

1704950.0

In [140]:
df_trade.head()

,SPY,AAPL,IBM,GOOG,XOM,Cash
2011-01-10,0.0,5000.0,0.0,0.0,0.0,-1704950.0
2011-01-11,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-12,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-13,0.0,-5000.0,4000.0,0.0,0.0,1142800.0
2011-01-14,0.0,0.0,0.0,0.0,0.0,0.0


In [141]:
df_holdings = pd.DataFrame().reindex_like(df_price).fillna(0)

for i in range(df_holdings.shape[0]): 

    if i == 0: 
        df_holdings.iloc[i] = df_trade.iloc[i]
        df_holdings.iloc[i]['Cash'] += start_val

    else: 
        df_holdings.iloc[i] = df_holdings.iloc[i - 1] + df_trade.iloc[i]

In [147]:
df_holdings.iloc[0]['Cash'] = df_trade.iloc[0]['Cash'] + start_val

0.0

In [152]:
start_val=1000000
df_holdings.iloc[0] = df_trade.iloc[0]
df_holdings.iloc[0]['Cash'] += start_val

In [153]:
df_holdings.head()

,SPY,AAPL,IBM,GOOG,XOM,Cash
2011-01-10,0.0,5000.0,0.0,0.0,0.0,-704950.0
2011-01-11,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-12,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-13,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-14,0.0,0.0,0.0,0.0,0.0,0.0


In [157]:
df_holdings.iloc[1] + df_holdings.iloc[0]

SPY          0.0
AAPL      5000.0
IBM          0.0
GOOG         0.0
XOM          0.0
Cash   -704950.0
dtype: float64

In [158]:
for i in range(df_holdings.shape[0]): 

    if i == 0: 
        df_holdings.iloc[i] = df_trade.iloc[i]
        df_holdings.iloc[i]['Cash'] += start_val

    else: 
        df_holdings.iloc[i] = df_holdings.iloc[i - 1] + df_trade.iloc[i]

In [162]:
df_trade.head()

,SPY,AAPL,IBM,GOOG,XOM,Cash
2011-01-10,0.0,5000.0,0.0,0.0,0.0,-1704950.0
2011-01-11,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-12,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-13,0.0,-5000.0,4000.0,0.0,0.0,1142800.0
2011-01-14,0.0,0.0,0.0,0.0,0.0,0.0


In [163]:
df_holdings.head()

,SPY,AAPL,IBM,GOOG,XOM,Cash
2011-01-10,0.0,5000.0,0.0,0.0,0.0,-704950.0
2011-01-11,0.0,5000.0,0.0,0.0,0.0,-704950.0
2011-01-12,0.0,5000.0,0.0,0.0,0.0,-704950.0
2011-01-13,0.0,0.0,4000.0,0.0,0.0,437850.0
2011-01-14,0.0,0.0,4000.0,0.0,0.0,437850.0


In [168]:
df_value = df_price * df_holdings

In [170]:
df_value.sum(axis=1)

2011-01-10    1000000.0
2011-01-11     995950.0
2011-01-12    1009800.0
2011-01-13    1016050.0
2011-01-14    1020650.0
2011-01-18    1023170.0
2011-01-19    1042730.0
2011-01-20    1043170.0
2011-01-21    1042010.0
2011-01-24    1058050.0
2011-01-25    1065090.0
2011-01-26    1063530.0
2011-01-27    1063940.0
2011-01-28    1040900.0
2011-01-31    1051110.0
2011-02-01    1067870.0
2011-02-02    1067830.0
2011-02-03    1066740.0
2011-02-04    1070010.0
2011-02-07    1074000.0
2011-02-08    1087410.0
2011-02-09    1081730.0
2011-02-10    1076990.0
2011-02-11    1084090.0
2011-02-14    1085300.0
2011-02-15    1079820.0
2011-02-16    1082050.0
2011-02-17    1086370.0
2011-02-18    1093550.0
2011-02-22    1062400.0
2011-02-23    1056590.0
2011-02-24    1056410.0
2011-02-25    1063510.0
2011-02-28    1065310.0
2011-03-01    1045230.0
2011-03-02    1045980.0
2011-03-03    1067710.0
dtype: float64